# ¿Cuánto deberían valer las propiedades en Milwaukee, Wisconsin? (Parte II)

## Objetivos

En los últimos casos, hemos visto varias técnicas utilizadas para diagnosticar la aplicabilidad de los modelos de regresión lineal.

## Introducción 

**Problema empresarial.** Su tarea es **crear un modelo para predecir los precios de las propiedades en la ciudad de Milwaukee, Wisconsin**.

**Contexto analítico.**  el conjunto de datos consta de ventas de propiedades (comerciales y residenciales) en Milwaukee, Wisconsin, de 2002 a 2018. 


# Preparar data

In [15]:
### Load relevant packages
import pandas                  as pd
import numpy                   as np
import matplotlib.pyplot       as plt
import seaborn                 as sns
import statsmodels.formula.api as smf
import statsmodels.api         as sm
import scipy

%matplotlib inline
plt.style.use('ggplot')
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [16]:
data = pd.read_csv("2002-2018-property-sales-data.csv",
    dtype = { # Categorias
        "PropType": "category",
        "District": "category",
        "Extwall": "category",
        "Nbhd": "category",
        "Style": "category",
    },
    parse_dates=["Sale_date"],
)
def remove_unused_categories(data):
    """ The `remove_unused_categories` method in pandas
        removes categories from a Series if there are no
        elements of that category.
        
        This function is a convenience function that removes
        unused categories for all categorical columns
        of a data frame.
        
        The reason this is useful is that when we
        fit a linear regression, `statsmodels` will
        create a coefficient for every category in a column,
        and so unused categories pollute the results.
    """
    for cname in data:
        col = data[cname]
        if pd.api.types.is_categorical_dtype(col):
            data[cname] = col.cat.remove_unused_categories()
    return data

clean = np.where(
    (data["Sale_price"] > 2000) & # Importante
    (data["Year_Built"] > 1800) &
    (data["Fin_sqft"] > 0) & # Condicion logica
    (data["Lotsize"] > 0)  & # Condicion Logica
    (data["PropType"] == "Residential")
    )
data_clean = data.iloc[clean].copy()
remove_unused_categories(data_clean).head()

/var/folders/wt/dd5nh3v12zvd92jwl2f9nt400000gn/T/ipykernel_79723/1550737232.py:27: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(col):


,PropType,Taxkey,Address,CondoProject,District,Nbhd,Style,Extwall,Stories,Year_Built,Nr_of_rms,Fin_sqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price
10,Residential,3080013000,3033 N 35TH ST,NaN,7,2960,AP 1,Frame,2.0,1913,0,3476,4,9,1,0,5040,2002-02-01,42000
51,Residential,3190434000,1908 E WEBSTER PL,NaN,3,3170,Rm or Rooming House,Frame,2.0,1897,0,1992,4,2,2,0,2880,2002-05-01,145000
67,Residential,3891722000,812 N 25TH ST,NaN,4,3040,Rm or Rooming House,Frame,2.0,1907,0,2339,6,0,1,0,3185,2002-06-01,30000
116,Residential,3880628000,959 N 34TH ST,NaN,4,2300,AP 1,Frame,2.0,1890,0,2329,4,4,1,0,5781,2002-10-01,66500
134,Residential,3880406000,3209 W WELLS ST,NaN,4,2300,Mansion,Stone,2.5,1891,0,7450,2,7,6,0,15600,2002-11-01,150500


Debemos dividir aleatoriamente los datos en un conjunto de entrenamiento y un conjunto de prueba. El **conjunto de entrenamiento** es aquel en el que entrenamos y ajustamos nuestro modelo de regresión lineal múltiple. Luego ejecutamos nuestro modelo ajustado en el **conjunto de prueba** y comparamos sus predicciones con los datos de la variable de respuesta del conjunto de prueba real para evaluar su rendimiento.

In [30]:
train=data_clean.iloc[0:19556,:]
test =data_clean.iloc[19556:,:]

In [25]:
from sklearn.model_selection import train_test_split
train, test= train_test_split(data_clean,test_size=0.2)
train

,PropType,Taxkey,Address,CondoProject,District,Nbhd,Style,Extwall,Stories,Year_Built,Nr_of_rms,Fin_sqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price
4151,Residential,6680143100,5858 S HONEY CREEK DR,NaN,13,4840,Duplex N/S,Aluminum / Vinyl,2.0,1959,0,2052,2,6,2,0,9125,2009-10-01,198000
4760,Residential,4210269000,603 S OAK PARK CT,NaN,10,2710,Milwaukee Bungalow,Aluminum / Vinyl,1.0,1926,0,982,1,3,1,0,5124,2009-12-01,46500
15660,Residential,4690202000,1623-A W BECHER ST,NaN,12,4120,Duplex O/S,Aluminum / Vinyl,1.5,1922,0,1153,2,4,2,0,3990,2014-10-01,59000
33849,Residential,2471212000,3715 W MARION ST,NaN,7,1680,Cape Cod,Stone,1.0,1952,0,984,1,3,1,0,5856,2018-11-01,71400
10673,Residential,2722824000,3536 N 11TH ST,NaN,6,2860,Residence O/S,Aluminum / Vinyl,1.5,1904,0,1549,1,4,1,0,3750,2013-02-01,15000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21291,Residential,5950511000,4421 S 5TH ST,NaN,13,4740,Cape Cod,Brick,1.0,1956,0,1317,1,3,1,0,6150,2016-05-01,137500
20138,Residential,1719808000,5665 N 40TH ST,NaN,1,900,Colonial,Aluminum / Vinyl,2.0,1932,0,1520,1,4,1,1,7080,2016-02-01,41000
12095,Residential,2611049000,3825 N 99TH ST,NaN,5,2100,Cape Cod,Aluminum / Vinyl,1.0,1954,0,1038,1,3,1,0,4840,2013-08-01,105000
8458,Residential,5730321000,3942 S 53RD ST,NaN,11,4420,Ranch,Aluminum / Vinyl,1.0,1954,0,933,1,3,1,0,5959,2011-12-01,105000


In [33]:
#### RESPUESTA AQUI
np.random.seed(135568109) 
ndata = len(data_clean)
print(len(data))
idx_train = np.random.choice(range(ndata),int(0.8*ndata),replace=False)
idx_test  = np.asarray(list(set(range(ndata)) - set(idx_train)))
train     = data_clean.iloc[idx_train] # train
test      = data_clean.iloc[idx_test]  # test
print(train.shape) # 19,312 
print(test.shape)  #  4,829 

34523
(19556, 19)
(4889, 19)


Ahora apodemos hacer un modelo para poder predecir `Sales` en funcion de `District + Units+Fin_sqft`

Analiza los resultados , hacen sentido?

In [34]:
#### RESPUESTA AQUI
model_log = smf.ols(formula = "Sale_price ~ District + Units+ Fin_sqft", 
                 data = train).fit()
model_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Sale_price   R-squared:                       0.674
Model:                            OLS   Adj. R-squared:                  0.674
Method:                 Least Squares   F-statistic:                     2522.
Date:                Thu, 30 Jan 2025   Prob (F-statistic):               0.00
Time:                        19:17:22   Log-Likelihood:            -2.3898e+05
No. Observations:               19556   AIC:                         4.780e+05
Df Residuals:                   19539   BIC:                         4.781e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       2.111e+04   2037.224     10.361      0.000    1.71e+04    2.51e+04
District[T.10]  4.993e+04   2053.420     24.318      0.000    4.59e+04     5.4e+04
District[T.11]  7.286e+04   1974.069     36.908      0.000     6.9e+04    7.67e+04
District[T.12] -3327.9414   3134.914     -1.062      0.288   -9472.640    2816.758
District[T.13]  7.162e+04   2044.727     35.025      0.000    6.76e+04    7.56e+04
District[T.14]  8.781e+04   2037.479     43.097      0.000    8.38e+04    9.18e+04
District[T.15] -4.997e+04   3003.689    -16.638      0.000   -5.59e+04   -4.41e+04
District[T.2]   1.867e+04   2275.581      8.206      0.000    1.42e+04    2.31e+04
District[T.3]   1.401e+05   2336.647     59.938      0.000    1.35e+05    1.45e+05
District[T.4]  -6.818e+04   4719.596    -14.446      0.000   -7.74e+04   -5.89e+04
District[T.5]   5.548e+04   1973.580     28.113      0.000    5.16e+04    5.94e+04
District[T.6]  -5797.9472   2753.277     -2.106      0.035   -1.12e+04    -401.289
District[T.7]  -1.421e+04   2442.523     -5.817      0.000    -1.9e+04   -9420.894
District[T.8]  -1088.2739   2637.765     -0.413      0.680   -6258.519    4081.971
District[T.9]   4.206e+04   2385.842     17.629      0.000    3.74e+04    4.67e+04
Units          -6.999e+04    943.776    -74.160      0.000   -7.18e+04   -6.81e+04
Fin_sqft          99.1541      0.723    137.224      0.000      97.738     100.570
==============================================================================
Omnibus:                    14105.557   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1397399.716
Skew:                           2.710   Prob(JB):                         0.00
Kurtosis:                      44.056   Cond. No.                     3.31e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.31e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Ahora intentemos realizar algunas transformaciones para entender si se mejoran nuestros resultados
Puedes usar esto como referencia para hacer el modelo 
```python
model_log = smf.ols(formula = "np.log(Sale_price) ~ District + Units"
                           "+ np.log(Fin_sqft)", 
                 data = train).fit()
```

In [38]:
#### RESPUESTA AQUI
model_log = smf.ols(formula = "np.log(Sale_price) ~ District + Units + np.log(Fin_sqft)", 
                 data = train).fit()
model_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     np.log(Sale_price)   R-squared:                       0.607
Model:                            OLS   Adj. R-squared:                  0.607
Method:                 Least Squares   F-statistic:                     1886.
Date:                Thu, 30 Jan 2025   Prob (F-statistic):               0.00
Time:                        19:26:56   Log-Likelihood:                -7612.5
No. Observations:               19556   AIC:                         1.526e+04
Df Residuals:                   19539   BIC:                         1.539e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            5.2685      0.063     84.177      0.000       5.146       5.391
District[T.10]       0.6211      0.015     41.545      0.000       0.592       0.650
District[T.11]       0.8147      0.014     56.736      0.000       0.787       0.843
District[T.12]      -0.0125      0.023     -0.546      0.585      -0.057       0.032
District[T.13]       0.7995      0.015     53.747      0.000       0.770       0.829
District[T.14]       0.8624      0.015     58.160      0.000       0.833       0.891
District[T.15]      -0.4609      0.022    -21.077      0.000      -0.504      -0.418
District[T.2]        0.3029      0.017     18.298      0.000       0.270       0.335
District[T.3]        1.0236      0.017     60.776      0.000       0.991       1.057
District[T.4]       -0.2284      0.034     -6.670      0.000      -0.296      -0.161
District[T.5]        0.6830      0.014     47.556      0.000       0.655       0.711
District[T.6]       -0.1563      0.020     -7.799      0.000      -0.196      -0.117
District[T.7]       -0.0693      0.018     -3.896      0.000      -0.104      -0.034
District[T.8]        0.1562      0.019      8.136      0.000       0.119       0.194
District[T.9]        0.5684      0.017     32.742      0.000       0.534       0.602
Units               -0.3476      0.007    -50.168      0.000      -0.361      -0.334
np.log(Fin_sqft)     0.8642      0.009     94.900      0.000       0.846       0.882
==============================================================================
Omnibus:                     4417.978   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            26736.926
Skew:                          -0.949   Prob(JB):                         0.00
Kurtosis:                       8.405   Cond. No.                         186.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Variables categoricas

Quizás haya notado que hay más de una docena de coeficientes para `Distrito` arriba. Esto se debe a que `Distrito` es una variable categórica y, para las características categóricas, se obtiene un coeficiente para todas las categorías excepto una. Si solo hay dos categorías (por ejemplo, género), esto es intuitivo: la característica se convierte en una columna de ceros y unos antes de introducirla en la regresión, donde se trata como una variable numérica regular.

Cuando hay más de dos categorías, una categoría se designa como la categoría de "referencia" o "línea de base", y se crean columnas "ficticias" de unos y ceros para todas las demás categorías. Tomemos un ejemplo ficticio con tres categorías y cinco filas:

| Categoría |
|--------------|
| A |
| B |
| C |
| C |
| A |

Antes de ajustar la regresión lineal, la columna `Categoría` se transforma en **dos** columnas “ficticias” (no se agrega una columna para la categoría de referencia).

La primera columna es 1 si el distrito es `B` y 0 en caso contrario, mientras que la segunda columna es 1 si el distrito es `C` y 0 en caso contrario. Obtenemos:

| Categoría_B | Categoría_C |
|--------------|-------------|
| 0 | 0 |
| 1 | 0 |
| 0 | 1 |
| 0 | 1 |
| 0 | 0 |

Las columnas ficticias se introducen en la regresión lineal y se tratan como variables numéricas regulares. Esta técnica se denomina **codificación one-hot** y se puede realizar manualmente con la función pandas `pd.get_dummies()`:

In [42]:
df_dummy=pd.DataFrame({'Category':['A','B','C','C','A']})
pd.get_dummies(df_dummy,columns=['Category'], drop_first=True) 

,Category_B,Category_C
0,False,False
1,True,False
2,False,True
3,False,True
4,False,False


Al igual que en el caso binario, la elección de la línea de base cambia los coeficientes y su interpretación; el coeficiente `Distrito[T.3]` de 1.0236 debe interpretarse como la diferencia en los resultados previstos entre los distritos 3 y 1. Pero la elección de la línea de base no afecta las predicciones ni el rendimiento del modelo, por lo que la mayoría del software elegirá arbitrariamente una categoría como línea de base sin previo aviso (a menudo la primera alfabéticamente).

Ahora hay un nuevo desafio 

Agrega la columna `Style` a nuestro último modelo de regresión. Hazlo de dos maneras: primero transformando la variable `Style` usando `pd.get_dummies()` y luego sin la transformación. Verifica que obtengas los mismos resultados. De acuerdo con este modelo, ¿qué propiedad style es la más deseable?

**Sugerencia:** La variable `Style` contiene nombres, como `Residence O/S`, que pueden ser problemáticos al escribir fórmulas `smf.ols` (consulta la [Sintaxis de Pasty](https://patsy.readthedocs.io/en/latest/index.html)). Puedes usar la sintaxis `Pasty` entre comillas `Q()` en los nombres de variables para evitar este problema:
```python
formula="np.log(Sale_price) ~Q("Residence O/S")+...
```

In [46]:
print(train.Style.unique())

['Ranch', 'Cape Cod', 'Residence O/S', 'Colonial', 'Duplex O/S', ..., 'Mansion', 'Cottage', 'Bi-Level', 'AP 1', 'Rm or Rooming House']
Length: 18
Categories (18, object): ['AP 1', 'Bi-Level', 'Cape Cod', 'Colonial', ..., 'Split Level', 'Townhouse', 'Triplex', 'Tudor']


In [49]:
#### RESPUESTA AQUI
# Ordenar Style
Styles=list(train['Style'].unique())
Styles.sort() #Just to keep track of the baseline category.

train_dummy=pd.get_dummies(train,columns=['Style'],prefix='',prefix_sep='',drop_first=True)

# Formula
formula0="Sale_price ~ "
for style in Styles[1:]:
    formula0+='Q("'+style+'")+'
formula0=formula0+'District + Units+ Fin_sqft'
print(formula0)
# Modelo
model_style_1 = smf.ols(formula = formula0, 
                 data = train_dummy).fit()
model_style_1.summary()

Sale_price ~ Q("Bi-Level")+Q("Cape Cod")+Q("Colonial")+Q("Cottage")+Q("Dplx Bungalow")+Q("Duplex N/S")+Q("Duplex O/S")+Q("Duplex-Cottage")+Q("Mansion")+Q("Milwaukee Bungalow")+Q("Ranch")+Q("Residence O/S")+Q("Rm or Rooming House")+Q("Split Level")+Q("Townhouse")+Q("Triplex")+Q("Tudor")+District + Units+ Fin_sqft


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Sale_price   R-squared:                       0.717
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     1502.
Date:                Thu, 30 Jan 2025   Prob (F-statistic):               0.00
Time:                        19:33:01   Log-Likelihood:            -2.3758e+05
No. Observations:               19556   AIC:                         4.752e+05
Df Residuals:                   19522   BIC:                         4.755e+05
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                        -1.412e+05   1.94e+04     -7.269      0.000   -1.79e+05   -1.03e+05
Q("Bi-Level")[T.True]             9.738e+04   1.72e+04      5.651      0.000    6.36e+04    1.31e+05
Q("Cape Cod")[T.True]             1.035e+05   1.61e+04      6.421      0.000    7.19e+04    1.35e+05
Q("Colonial")[T.True]             1.183e+05   1.61e+04      7.327      0.000    8.67e+04     1.5e+05
Q("Cottage")[T.True]              7.887e+04   1.63e+04      4.849      0.000     4.7e+04    1.11e+05
Q("Dplx Bungalow")[T.True]        2.047e+04   1.34e+04      1.531      0.126   -5740.712    4.67e+04
Q("Duplex N/S")[T.True]           3.929e+04   1.34e+04      2.933      0.003     1.3e+04    6.55e+04
Q("Duplex O/S")[T.True]           1.995e+04   1.33e+04      1.495      0.135   -6208.769    4.61e+04
Q("Duplex-Cottage")[T.True]       1.084e+04   1.39e+04      0.779      0.436   -1.64e+04    3.81e+04
Q("Mansion")[T.True]              3.404e+05   1.72e+04     19.833      0.000    3.07e+05    3.74e+05
Q("Milwaukee Bungalow")[T.True]   8.773e+04   1.61e+04      5.438      0.000    5.61e+04    1.19e+05
Q("Ranch")[T.True]                1.118e+05   1.61e+04      6.933      0.000    8.02e+04    1.43e+05
Q("Residence O/S")[T.True]        9.171e+04   1.61e+04      5.695      0.000    6.01e+04    1.23e+05
Q("Rm or Rooming House")[T.True] -1.774e+04   2.06e+04     -0.861      0.389   -5.81e+04    2.26e+04
Q("Split Level")[T.True]           9.36e+04   1.66e+04      5.623      0.000     6.1e+04    1.26e+05
Q("Townhouse")[T.True]            2.356e+04   1.38e+04      1.707      0.088   -3487.772    5.06e+04
Q("Triplex")[T.True]             -1.038e+04   1.18e+04     -0.881      0.378   -3.35e+04    1.27e+04
Q("Tudor")[T.True]                1.447e+05   1.64e+04      8.798      0.000    1.12e+05    1.77e+05
District[T.10]                    5.663e+04   1936.423     29.244      0.000    5.28e+04    6.04e+04
District[T.11]                    6.786e+04   1856.553     36.551      0.000    6.42e+04    7.15e+04
District[T.12]                    9426.9151   3095.539      3.045      0.002    3359.394    1.55e+04
District[T.13]                    6.844e+04   1912.556     35.785      0.000    6.47e+04    7.22e+04
District[T.14]                    9.647e+04   1939.902     49.731      0.000    9.27e+04       1e+05
District[T.15]                    -3.62e+04   2857.201    -12.668      0.000   -4.18e+04   -3.06e+04
District[T.2]                     1.358e+04   2144.492      6.334      0.000    9379.169    1.78e+04
District[T.3]                     1.465e+05   2280.762     64.252      0.000    1.42e+05    1.51e+05
District[T.4]                    -6.435e+04   4478.078    -14.370      0.000   -7.31e+04   -5.56e+04
District[T.5]                     5.162e+04   1859.15

In [47]:
model_style = smf.ols(formula = "Sale_price ~ Style + District + Units+ Fin_sqft", 
                 data = train).fit()
model_style.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Sale_price   R-squared:                       0.717
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     1502.
Date:                Thu, 30 Jan 2025   Prob (F-statistic):               0.00
Time:                        19:30:54   Log-Likelihood:            -2.3758e+05
No. Observations:               19556   AIC:                         4.752e+05
Df Residuals:                   19522   BIC:                         4.755e+05
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                    -1.412e+05   1.94e+04     -7.269      0.000   -1.79e+05   -1.03e+05
Style[T.Bi-Level]             9.738e+04   1.72e+04      5.651      0.000    6.36e+04    1.31e+05
Style[T.Cape Cod]             1.035e+05   1.61e+04      6.421      0.000    7.19e+04    1.35e+05
Style[T.Colonial]             1.183e+05   1.61e+04      7.327      0.000    8.67e+04     1.5e+05
Style[T.Cottage]              7.887e+04   1.63e+04      4.849      0.000     4.7e+04    1.11e+05
Style[T.Dplx Bungalow]        2.047e+04   1.34e+04      1.531      0.126   -5740.712    4.67e+04
Style[T.Duplex N/S]           3.929e+04   1.34e+04      2.933      0.003     1.3e+04    6.55e+04
Style[T.Duplex O/S]           1.995e+04   1.33e+04      1.495      0.135   -6208.769    4.61e+04
Style[T.Duplex-Cottage]       1.084e+04   1.39e+04      0.779      0.436   -1.64e+04    3.81e+04
Style[T.Mansion]              3.404e+05   1.72e+04     19.833      0.000    3.07e+05    3.74e+05
Style[T.Milwaukee Bungalow]   8.773e+04   1.61e+04      5.438      0.000    5.61e+04    1.19e+05
Style[T.Ranch]                1.118e+05   1.61e+04      6.933      0.000    8.02e+04    1.43e+05
Style[T.Residence O/S]        9.171e+04   1.61e+04      5.695      0.000    6.01e+04    1.23e+05
Style[T.Rm or Rooming House] -1.774e+04   2.06e+04     -0.861      0.389   -5.81e+04    2.26e+04
Style[T.Split Level]           9.36e+04   1.66e+04      5.623      0.000     6.1e+04    1.26e+05
Style[T.Townhouse]            2.356e+04   1.38e+04      1.707      0.088   -3487.772    5.06e+04
Style[T.Triplex]             -1.038e+04   1.18e+04     -0.881      0.378   -3.35e+04    1.27e+04
Style[T.Tudor]                1.447e+05   1.64e+04      8.798      0.000    1.12e+05    1.77e+05
District[T.10]                5.663e+04   1936.423     29.244      0.000    5.28e+04    6.04e+04
District[T.11]                6.786e+04   1856.553     36.551      0.000    6.42e+04    7.15e+04
District[T.12]                9426.9151   3095.539      3.045      0.002    3359.394    1.55e+04
District[T.13]                6.844e+04   1912.556     35.785      0.000    6.47e+04    7.22e+04
District[T.14]                9.647e+04   1939.902     49.731      0.000    9.27e+04       1e+05
District[T.15]                -3.62e+04   2857.201    -12.668      0.000   -4.18e+04   -3.06e+04
District[T.2]                 1.358e+04   2144.492      6.334      0.000    9379.169    1.78e+04
District[T.3]                 1.465e+05   2280.762     64.252      0.000    1.42e+05    1.51e+05
District[T.4]                -6.435e+04   4478.078    -14.370      0.000   -7.31e+04   -5.56e+04
District[T.5]                 5.162e+04   1859.156     27.765      0.000     4.8e+04    5.53e+04
District[T.6]                 9889.3671   2662.832      3.714      0.000    

# Evaluacion de resultados

In [63]:
C=pd.DataFrame()
C["Model"]=model_style.predict(test)
C["Reales"]=test.Sale_price
C

,Model,Reales
12700,141226.087131,159000
23868,156397.616608,135000
135,110702.087393,75000
12705,231492.922294,350000
12706,177467.248014,174900
...,...,...
23856,134901.529625,142000
23858,123137.144941,143900
23859,169513.775705,174500
23864,195129.590798,250400


In [65]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error,max_error

print("MAE:",mean_absolute_error(y_true=C["Reales"], y_pred=C["Model"]))
print("MSE:", mean_squared_error(y_true=C["Reales"], y_pred=C["Model"]))
print("R2:", r2_score(y_true=C["Reales"], y_pred=C["Model"]))
print("MedAbsError:",median_absolute_error(y_true=C["Reales"], y_pred=C["Model"]))
print("Max Error:", max_error(y_true=C["Reales"], y_pred=C["Model"]))

MAE: 30102.90310659643
MSE: 1816100273.3168392
R2: 0.7096714568445633
MedAbsError: 22691.309518094553
Max Error: 515423.4667877769
